# Continuous Control Second Version (DDPG)

---

In this notebook, you will learn how to use Unity ML-Agents environment for the second project (second version) of the **Deep Reinforcement Learning Nanodegree program**.

### 1. Start the Environment

Please refer to the file `README.md` for environment preparation.

Firstly, we begin by importing the necessary packages.

After installation, we begin by importing the necessary packages.

In [1]:
import random
import copy
from collections import deque, namedtuple, OrderedDict

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from unityagents import UnityEnvironment


Now, we will start the environment. 

Please change the `file_name` parameter to match the location of the Unity Environment that you downloaded

In [2]:
# select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64', no_graphics=True)

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents, env_info.vector_observations.shape[0])

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

print(states[0].shape, states[0,:].shape)

Number of agents: 20 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]
(33,) (33,)


### 3. Training the Agent (using DDPG)
Train the agent to solve the environment.

The `Agent` contains 4 models: `actor_local`, `actor_target`, `critic_local`, `critic_target`. 

The **Actor model** structure are as follows:
![actor_model](resources/actor_model.png)


The **Critic model** structure are as follows:
![critic_model](resources/critic_model.png)

The following are training codes.

In [5]:
from ddpg_agent import Agent

# ddpg function for single and multiple agents
def ddpg(n_episodes=100,
         eps_start=1.0, eps_decay=1e-5, eps_end=0.05,
         max_t=10000, learn_every_step=100):
    scores_deque = deque(maxlen=100)
    scores, actor_losses, critic_losses = [], [], []
    eps = eps_start
    for i_episode in range(1, 1 + n_episodes):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations

        agent.reset()

        avg_score = 0
        actor_loss_list, critic_loss_list = [], []
        for t in range(max_t):
            actions = agent.act(states, add_noise=random.random() < eps)
            eps = max(eps - eps_decay, eps_end)

            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done

            agent.step(states, actions, rewards, next_states, dones)

            # Learn, if enough samples are available in memory
            if len(agent.memory) > agent.batch_size and \
                    t % learn_every_step == 0:
                for _ in range(3):
                    experiences = agent.memory.sample(batch_size=agent.batch_size)
                    actor_loss, critic_loss = agent.learn(experiences, agent.gamma,
                                                          last_action_loss=actor_loss_list[-1] if actor_loss_list else None)
                    actor_loss_list.append(actor_loss)
                    critic_loss_list.append(critic_loss)

            avg_score += np.mean(rewards)
            states = next_states
            if np.any(dones):
                break

        scores_deque.append(avg_score)
        scores.append(avg_score)
        actor_losses.append(np.mean(actor_loss_list))
        critic_losses.append(np.mean(critic_loss_list))
        print(f"\rEpisode {i_episode}\tExploration: {eps:.6f}\t"
              f"Average Score: {np.mean(scores_deque):.2f}\tCurrent Score: {avg_score:.2f}\t"
              f"Actor Loss: {np.mean(actor_loss_list):.2e}\tCritic Loss: {np.mean(critic_loss_list):.2e}")

        if i_episode % 100 == 0:
            # agent.save()
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))

        if np.mean(scores_deque) > 30 and len(scores_deque) >= 100:
            agent.save()
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            break

    return scores, actor_losses, critic_losses

In [6]:
# Define global parameters

SEED = 1

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


ACTOR_LR = 1e-4  # learning rate
ACTOR_HIDDEN_UNITS = (256, 128)
ACTOR_WEIGHT_DECAY = 1e-5

CRITIC_LR = 1e-4  # learning rate
CRITIC_HIDDEN_UNITS = (256, 128)
CRITIC_WEIGHT_DECAY = 1e-5

MEMORY_BUFFER_SIZE = int(1e6)  # maximum size of replay buffer
BATCH_SIZE = 128  # mini-batch size
GAMMA = 0.99  # discount factor
TAU = 1e-3  # interpolation parameter

N_EPISODES = 2000  # total episodes to train
EPS_START = 1.0  # initial value for exploration (epsilon)
EPS_DECAY = 2e-5  # epsilon decay value after each step
EPS_END = 0.05  # lower limit of epsilon
MAX_STEPS = 10000  # maximum training steps of each epoch
LEARN_EVERY_STEP = 5  # extra learning after every step


In [7]:
agent = Agent(state_size=state_size,
              action_size=action_size,
              num_agents=num_agents,
              seed=SEED, buffer_size=MEMORY_BUFFER_SIZE,
              actor_lr=ACTOR_LR, actor_hidden_sizes=ACTOR_HIDDEN_UNITS, actor_weight_decay=ACTOR_WEIGHT_DECAY,
              critic_lr=CRITIC_LR, critic_hidden_sizes=CRITIC_HIDDEN_UNITS, critic_weight_decay=CRITIC_WEIGHT_DECAY,
              batch_size=BATCH_SIZE, gamma=GAMMA, tau=TAU)
#print(agent)

In [8]:
# Start training
import matplotlib.pyplot as plt

print(agent)

scores, actor_losses, critic_losses = ddpg(n_episodes=N_EPISODES,
                                           eps_start=EPS_START, eps_decay=EPS_DECAY, eps_end=EPS_END,
                                           max_t=MAX_STEPS, learn_every_step=LEARN_EVERY_STEP)

agent.save()

fig = plt.figure()
ax1 = fig.add_subplot(311)
ax1.plot(np.arange(1, len(scores) + 1), scores)
ax1.set_ylabel('Score')
ax1.set_xlabel('Episode #')

ax2 = fig.add_subplot(312)
ax2.plot(np.arange(1, len(actor_losses) + 1), actor_losses)
# ax2.legend()
ax2.set_ylabel('Actor Loss')
ax2.set_xlabel('Episode #')

ax3 = fig.add_subplot(313)
ax3.plot(np.arange(1, len(critic_losses) + 1), critic_losses)
ax3.set_ylabel('Critic Loss')
ax3.set_xlabel('Episode #')

plt.show()

Actor(
  (fc_body): Sequential(
    (0): Linear(in_features=33, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01, inplace)
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): LeakyReLU(negative_slope=0.01, inplace)
    (4): Linear(in_features=128, out_features=4, bias=True)
    (5): Tanh()
  )
)
Critic(
  (fc_body): Sequential(
    (0): Linear(in_features=33, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01, inplace)
  )
  (critic_body): Sequential(
    (0): Linear(in_features=260, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.01, inplace)
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
)
Episode 1	Exploration: 0.979980	Average Score: 0.54	Current Score: 0.54	Actor Loss: -9.74e-03	Critic Loss: 1.68e-05
Episode 2	Exploration: 0.959960	Average Score: 0.70	Current Score: 0.86	Actor Loss: -1.20e-02	Critic Loss: 2.52e-05
Episode 3	Exploration: 0.939940	Average Score: 0.76	Current Score: 0.90	Actor Lo

Episode 65	Exploration: 0.050000	Average Score: 21.85	Current Score: 34.27	Actor Loss: -2.06e+00	Critic Loss: 6.07e-03
Episode 66	Exploration: 0.050000	Average Score: 22.06	Current Score: 35.94	Actor Loss: -2.11e+00	Critic Loss: 5.63e-03
Episode 67	Exploration: 0.050000	Average Score: 22.28	Current Score: 36.63	Actor Loss: -2.15e+00	Critic Loss: 6.14e-03
Episode 68	Exploration: 0.050000	Average Score: 22.48	Current Score: 36.01	Actor Loss: -2.20e+00	Critic Loss: 5.67e-03
Episode 69	Exploration: 0.050000	Average Score: 22.68	Current Score: 36.51	Actor Loss: -2.24e+00	Critic Loss: 6.80e-03
Episode 70	Exploration: 0.050000	Average Score: 22.87	Current Score: 36.16	Actor Loss: -2.29e+00	Critic Loss: 6.66e-03
Episode 71	Exploration: 0.050000	Average Score: 23.06	Current Score: 36.37	Actor Loss: -2.33e+00	Critic Loss: 6.52e-03
Episode 72	Exploration: 0.050000	Average Score: 23.23	Current Score: 34.92	Actor Loss: -2.37e+00	Critic Loss: 6.71e-03
Episode 73	Exploration: 0.050000	Average Score: 

We extract the values from the log information above and rebuild the training plot as follows. (Refer to the file `multi_robots_plot.py`)

![training_img](resources/Continuous_Control_DDPG_20_robots.png)

From the plot above, the episode is **111** when average score of last 100 episodes larger than 30.

### 4. Watch the smart Agent to play

We load the actor model's parameters to watch the smart agent.

**Attention:** The pretrained checkpoints are on GPU machine. There will be an error if you load the checkpoints on non GPU machines, you should do some small changes in Agent#load method in `ddpg_agent.py`. Please refer to [this link](https://pytorch.org/tutorials/beginner/saving_loading_models.html) for hadling the issue.

In [9]:
agent.load()

env_info = env.reset(train_mode=True)[brain_name]
states = env_info.vector_observations

t_step, total_rewards = 0, 0
while True:
    actions = agent.act(states, add_noise=False)

    env_info = env.step(actions)[brain_name]
    next_states = env_info.vector_observations
    rewards = env_info.rewards
    dones = env_info.local_done

    states = next_states
    t_step += 1
    total_rewards += np.mean(rewards)
    if np.any(dones):
        break

print(f"Total Steps: {t_step}, Total Rewards: {total_rewards}")

Total Steps: 1001, Total Rewards: 37.376999164559294


### 5. Close the Unity Environment

In [10]:
env.close()